In [ ]:
# The conda env used in this notebook is a customised "crc-atlas env.: contains scanpy_helper_submodule from the LUCAS project
import os

import decoupler as dc
import numpy as np
import pandas as pd
import sc_atlas_helpers as ah
import scanpy as sc
from matplotlib.pyplot import rc_context
from scanpy_helper_submodule import scanpy_helpers as sh

# Figure default parameters
sc.settings.set_figure_params(
    dpi=400,
    facecolor="white",
    frameon=False,
)

# Set the number of max CPUs to be used by the processes
from threadpoolctl import threadpool_limits

cpus = 4
threadpool_limits(cpus)

### 1. Neutrophil cluster annotation - MUI dataset only (exluding AbSeq and Zurich data)

In [ ]:
# This h5ad was produced by Valentin after integrating the datasets with var. genes = 100
adata = sc.read_h5ad(
    "/data/projects/2022/CRCA/results/v1/artifacts/build_atlas/load_datasets/MUI_Innsbruck_neutro_hvg100.h5ad"
)

In [ ]:
# Show counts for sanity check (there are artifacts in the object - need to be filtered out: see below)
adata.obs.cell_type_coarse.value_counts()

In [ ]:
# Plot the UMAP plot to check initial clusters
# sc.set_figure_params(figsize=(6, 6), dpi=300)
sc.pl.umap(
    adata,
    color=["leiden_1.00", "patient_id", "sample_type"],
    layer="log1p_norm",
    vmin=0,
    vmax="p99",
    sort_order=False,
    frameon=False,
    add_outline=True,
    # legend_loc="on data"
    save="_all_neutro.png",
)

### 1.1 Subset and clean-up

In [ ]:
# Subset for Neutrophil cells only
adata_neutro_mui = adata[adata.obs["cell_type_coarse"].isin(["Neutrophil"])]

In [ ]:
# Keep only blood-tumor (for the DE analysis)
adata_neutro_mui = adata_neutro_mui[
    adata_neutro_mui.obs["sample_type"].isin(["blood", "tumor"])
]
# Keep only the MUI dataset (exclude AbSeq and Zurich datasets)
adata_neutro_mui = adata_neutro_mui[
    adata_neutro_mui.obs["dataset"].isin(["MUI_Innsbruck"])
]

In [ ]:
# Re-process the subsetted dataset
ah.pp.reprocess_adata_subset_scvi(adata_neutro_mui, leiden_res=1.5, n_neighbors=12)

In [ ]:
# Plot the re-processed dataset
# Note: The default figure params are changed here (I find these params produce more "readable" plots)
# Feel free to change
sc.set_figure_params(figsize=(6, 6), dpi=400)

# The actual plotting function
sc.pl.umap(
    adata_neutro_mui,
    color=["leiden", "sample_type"],
    layer="log1p_norm",
    vmin=0,
    vmax="p99",
    sort_order=False,
    frameon=False,
    add_outline=True,
    # legend_loc='on data',
    save="_MUI_only_neutro.png",
)

In [ ]:
# Plot the QC metrics
ah.pl.umap_qc_metrics(
    adata_neutro_mui,
    vmax_total_counts=10000,
    vmax_n_genes_by_counts=5000,
    save_dir="./figures/",
    prefix="MUI_only_neutro_qc.png",
)

### 1.2 Subset the Tumor samples (in order to re-process and annoate)

In [ ]:
# Subset for tunor
adata_neutro_mui_tumor = adata_neutro_mui[
    adata_neutro_mui.obs["sample_type"].isin(["tumor"])
]

In [ ]:
# Re-process
ah.pp.reprocess_adata_subset_scvi(adata_neutro_mui_tumor, leiden_res=0.3, n_neighbors=6)

In [ ]:
# Plot the leiden clusters
sc.pl.umap(
    adata_neutro_mui_tumor,
    color=["leiden"],
    layer="log1p_norm",
    vmin=0,
    vmax="p99",
    sort_order=False,
    frameon=False,
    cmap="Reds",
    add_outline=True,
    # legend_loc='on data',
    # save="_MUI_only_neutro_markers_2.png"
)

In [ ]:
# Rename the "cell_type_coarse" obs with the Tumor Ascossiated Neutrophil clusters (TAN)
cluster_annot = {
    "TAN1": [0],
    "TAN2": [1],
    "TAN3": [2, 3, 4],
}
cluster_annot = {str(vi): k for k, v in cluster_annot.items() for vi in v}

adata_neutro_mui_tumor.obs["cell_type_coarse"] = adata_neutro_mui_tumor.obs[
    "leiden"
].map(cluster_annot)

In [ ]:
# Plot the new TAN clusters
sc.pl.umap(
    adata_neutro_mui_tumor,
    color=["cell_type_coarse"],
    layer="log1p_norm",
    vmin=0,
    vmax="p99",
    sort_order=False,
    frameon=False,
    cmap="Reds",
    add_outline=True,
    legend_loc="on data",
    # save="_MUI_only_neutro_markers_2.png"
)

In [ ]:
# Integrate the new TAN clusters to the initial adata obj.
ah.pp.integrate_back(
    adata_neutro_mui, adata_neutro_mui_tumor, variable="cell_type_coarse"
)

### 1.3 Subset the Blood samples (in order to re-process and annoate)

In [ ]:
# Subset for blood
adata_neutro_mui_blood = adata_neutro_mui[
    adata_neutro_mui.obs["sample_type"].isin(["blood"])
]

In [ ]:
# Re-process
ah.pp.reprocess_adata_subset_scvi(adata_neutro_mui_blood, leiden_res=0.3, n_neighbors=8)

In [ ]:
# Plot the leiden clusters
sc.pl.umap(
    adata_neutro_mui_blood,
    color=["leiden"],
    layer="log1p_norm",
    vmin=0,
    vmax="p99",
    sort_order=False,
    frameon=False,
    cmap="Reds",
    add_outline=True,
    legend_loc="on data",
    # save="_MUI_only_neutro_markers_2.png"
)

In [ ]:
# Rename the "cell_type_coarse" obs with the Blood Neutrophil clusters (BN)
cluster_annot = {
    "BN1": [2],
    "BN2": [1],
    "BN3": [3],
    "BN4": [0, 4],
}
cluster_annot = {str(vi): k for k, v in cluster_annot.items() for vi in v}

adata_neutro_mui_blood.obs["cell_type_coarse"] = adata_neutro_mui_blood.obs[
    "leiden"
].map(cluster_annot)

In [ ]:
# Plot the new BN clusters
sc.pl.umap(
    adata_neutro_mui_blood,
    color=["cell_type_coarse"],
    layer="log1p_norm",
    vmin=0,
    vmax="p99",
    sort_order=False,
    frameon=False,
    cmap="Reds",
    add_outline=True,
    legend_loc="on data",
    # save="_MUI_only_neutro_markers_2.png"
)

In [ ]:
# Integrate the new BN clusters to the initial adata obj.
ah.pp.integrate_back(
    adata_neutro_mui, adata_neutro_mui_blood, variable="cell_type_coarse"
)

### 1.4 Clean up and final annotation

In [ ]:
# Plot the leiden and the neutrophil (TAN/BN) annotation
sc.pl.umap(
    adata_neutro_mui,
    color=["leiden", "cell_type_coarse"],
    layer="log1p_norm",
    vmin=0,
    vmax="p99",
    sort_order=False,
    frameon=False,
    cmap="Reds",
    add_outline=True,
    legend_loc="on data",
    save="_MUI_only_neutro_final_anno.png",
)

In [ ]:
# Cluster 19 seems to belong to BN3 rather to BN4, so we will merge it with BN3
adata_neutro_mui_tmp = adata_neutro_mui[adata_neutro_mui.obs["leiden"].isin(["19"])]
cluster_annot = {
    "BN3": [19],
}
cluster_annot = {str(vi): k for k, v in cluster_annot.items() for vi in v}

adata_neutro_mui_tmp.obs["cell_type_coarse"] = adata_neutro_mui_tmp.obs["leiden"].map(
    cluster_annot
)
ah.pp.integrate_back(
    adata_neutro_mui, adata_neutro_mui_tmp, variable="cell_type_coarse"
)

In [ ]:
# Plot the final annotation for sanity check
sc.pl.umap(
    adata_neutro_mui,
    color=["leiden", "cell_type_coarse", "patient_id"],
    layer="log1p_norm",
    vmin=0,
    vmax="p99",
    sort_order=False,
    frameon=False,
    cmap="Reds",
    add_outline=True,
    legend_loc="on data",
    save="_MUI_only_neutro_final_anno.png",
)

In [ ]:
# Do not run - save the final adata
adata_neutro_mui.write_h5ad("../data/MUI_only_neutro_latest.h5ad")

### 2. Pseudobulk and DE analysis

In [ ]:
# Do not run - load in the Neutro dataset (produced above)
# adata_neutro_mui = sc.read_h5ad(
#     "../data/MUI_only_neutro_latest.h5ad"
# )

### 2.1 Prepare input for DESeq2 analysis - Between Tumor/Blood Neutrophils

In [ ]:
# Copy adata (for "safety"/idiotproofing reasons
adata_pseudo = adata_neutro_mui.copy()

In [ ]:
# merge comlumns - to be used for covariate formula
adata_pseudo.obs["sample_id"] = (
    adata_neutro_mui.obs["dataset"].astype(str)
    + "."
    + adata_neutro_mui.obs["sample_id"].astype(str)
)
adata_pseudo.obs["patient_id"] = (
    adata_neutro_mui.obs["study_id"].astype(str)
    + "."
    + adata_neutro_mui.obs["patient_id"].astype(str)
)

In [ ]:
# create the "counts" layer - needed for the DE analysis
adata_pseudo.layers["counts"] = adata_pseudo.X

In [ ]:
# create the "cell_type" layer - needed for psudobulking
adata_pseudo.obs["cell_type"] = "Neutrophil"

In [ ]:
# Keep the protein-coding only genes
adata_pseudo = adata_pseudo[:, adata.var["Class"].isin(["protein_coding"])].copy()

In [ ]:
# Create the variables to be used in the loop below - these should be taken care through Valentin's NextFlow implimentation
groups_col = "cell_type"
ids = "tumor_blood"
sample_col = "sample_id"
layer = "counts"

In [ ]:
# Pseudobulk and produce the CSV files - to be used for the DESeq2 analysis
pseudobulk = [
    (
        group,
        dc.get_pseudobulk(
            adata_pseudo[adata_pseudo.obs[groups_col] == group],
            sample_col=sample_col,
            groups_col=groups_col,
            layer=layer,
            mode="sum",
            min_prop=0.2,
            min_cells=40,
            min_counts=500,
            min_smpls=3,
        ),
    )
    for group in adata_pseudo.obs[groups_col].unique()
]

for group, pdata in pseudobulk:
    pdata.var_names.name = "gene_id"

    colData = pdata.obs
    colData.index.name = "sample"
    colData.to_csv(f"./tables/{ids}_{group}_colData.csv")

    rowData = pdata.var[["Geneid", "GeneSymbol", "Chromosome", "Class", "Length"]]
    rowData.to_csv(f"./tables/{ids}_{group}_rowData.csv")

    count_mat = pdata.to_df().T
    count_mat.index.name = "gene_id"
    # count_mat['gene_name'] = rowData.GeneSymbol
    count_mat.to_csv(f"./tables/{ids}_{group}_count_mat.csv")

#### 2.2 AUC - DE between clusters

In [ ]:
# Create the pseudobulk - per Neutrophil cluster
pb_n = sh.pseudobulk.pseudobulk(
    adata_pseudo, groupby=["patient_id", "cell_type_coarse"]
)

In [ ]:
# Run the analysis for the TAN/BN clusters
column = "cell_type_coarse"  # In order to use the correct clusters

for ct in pb_n.obs[column].unique():
    sh.signatures.fold_change(
        pb_n, obs_col=column, positive_class=ct, key_added=f"{ct}_fc"
    )
    sh.signatures.specific_fold_change(
        pb_n, obs_col=column, positive_class=ct, key_added=f"{ct}_sfc"
    )
    sh.signatures.roc_auc(
        pb_n, obs_col=column, positive_class=ct, key_added=f"{ct}_auroc"
    )

In [ ]:
markers = {
    ct: pb_n.var.loc[
        lambda x: (x[f"{ct}_auroc"] >= 0.7) & (x[f"{ct}_fc"] > 1) & (x[f"{ct}_sfc"] > 0)
    ]
    .sort_values(f"{ct}_auroc", ascending=False)
    .index.tolist()
    for ct in sorted(pb_n.obs[column].unique())
}

In [ ]:
# scale and store results in layer
pb_n.layers["scaled"] = sc.pp.scale(pb_n, copy=True).X

In [ ]:
annotation = "Neutro_sub"
n_genes = [5, 10, 20]
# sufixes = ["",".svg",".png"]
sufixes = [".png"]

In [ ]:
# Plot the results
for n in n_genes:
    for i in sufixes:
        sc.pl.matrixplot(
            pb_n,
            groupby=column,
            colorbar_title="mean z-score",
            var_names={k: v[:n] for k, v in markers.items()},
            layer="scaled",
            vmin=-2,
            vmax=2,
            cmap="bwr",
            save=f"{annotation}_{str(n)}_genes_auc{i}",
            show=False,
        )
        sh.signatures.plot_markers(
            pb_n,
            groupby=column,
            markers={k: v[:n] for k, v in markers.items()},
            layer="scaled",
            top=n,
            save=f"sh_{annotation}_{str(n)}_genes_auc{i}",
            show=False,
        )
        # sh.signatures.plot_metric_strip(
        #     pb_n, markers={k: v[:n] for k, v in markers.items()}, top=n
        # )

In [ ]:
# for n in n_genes:
#     sh.signatures.plot_metric_strip(
#         pb_n, markers={k: v[:5] for k, v in markers.items()}, top=5
#     )

In [ ]:
sh.signatures.plot_metric_strip(
    pb_n, markers={k: v[:5] for k, v in markers.items()}, top=5
)

In [ ]:
sh.signatures.plot_metric_strip(
    pb_n, markers={k: v[:10] for k, v in markers.items()}, top=10
)

In [ ]:
sh.signatures.plot_metric_strip(
    pb_n, markers={k: v[:20] for k, v in markers.items()}, top=20
)